### Introduction

This is an introduction script for using dimer calculations to calculate the preferred packing motifs between molecules. The use cases of this code have been for cages thus far, and therefore the tutorial is set out in terms of cages, but the code can be used to calculate the interactions between any molecules in theory

In [1]:
import stk
import dimer_calculations
from dimer_calculations import Optimiser_functions
from dimer_calculations import utils
from dimer_calculations import cage
from dimer_calculations import dimer_centroids
from dimer_calculations import pores
from rdkit import Chem
import os
import py3Dmol
from rdkit.Chem import AllChem as rdkit


In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}},
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

def mol_with_vec(stk_mol,vectors,magnitude):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}},
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    for i in range(len(vectors)):
        x=magnitude*vectors[i,0]
        y=magnitude*vectors[i,1]
        z=magnitude*vectors[i,2]

        p.addArrow({
        'start': {'x': 0, 'y': 0, 'z': 0},  # Starting coordinates
        'end': {'x': x, 'y': y, 'z': z},    # Ending coordinates
        'radius': 0.2,                      # Arrow thickness
        'color': 'red'                      # Color of the arrow
        })
    p.zoomTo()
    p.show()

def check_catenation(filename,cage_molecule,threshold=0.1):

    cage_dimer = stk.BuildingBlock.init_from_file(filename)
    centroids = pores.get_centroids(cage_dimer)
    dimer = pores.two_pores(cage_dimer, centroids[0], centroids[1])
    pore_cage = pores.one_pore(cage_molecule)
    cutoff = threshold #if packing_2=='arene' else threshold*2
    catenated = pores.check_catenane(pore_cage, dimer, cutoff)
    return catenated



Load the molecule that you want to calculate dimers between using stk. This molecules is made out of the imine condensations of a diamine (Smile string NCCN) and a trialdehyde (Smile string O=Cc1cc(C=O)cc(C=O)c1)

In [3]:
filename='G17.mol'
molecule=stk.BuildingBlock.init_from_file(filename)
molecule=molecule.with_centroid([0,0,0])
show_stk_mol(molecule)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now we have the molecule we would like to take two of the molecules and displace them relative to eachother. This is so we can calculate the energy of the different placement of the molecules. For cages we are interested in doing these along the different "high symmetry" axes. In this case, for this cage, the high symmetry axes are between the centroid and the window, arene, and vertex. 

We can get the vertices by putting in the smiles (BySmiles) or Smarts (BySmarts) string of the moelcular component along the vertices

In [4]:
list_of_vertices,vertice_size = Optimiser_functions.Axes.BySmiles(molecule,smiles_string=utils.remove_aldehyde('NCCN'))
facet_axes,centroid_to_facet =Optimiser_functions.Axes.ByMidpoint(molecule,
    vectors=list_of_vertices,
    vertice_size=vertice_size,
    no_vectors_define_facet=3,
    tolerance=0.1)
list_of_arenes,centroid_to_arene = Optimiser_functions.Axes.BySmiles(molecule,smiles_string=utils.remove_aldehyde('O=Cc1cc(C=O)cc(C=O)c1'))
list_of_windows =Optimiser_functions.Axes.RemoveCommon(molecule,facet_axes, list_of_arenes)
centroid_to_window=centroid_to_arene


For the cage the different high symmetry axes and how to get them are defined above. Play around below with viewing those vectors and see if you can understand how and why they're defined that way.

In [5]:
mol_with_vec(molecule,facet_axes,centroid_to_window)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now lets look at a new molecule with a different topology. For this molecule, figure out (1) what the high symmetry axes are and (2) the vectors that you would need to define these axes

In [6]:
filename='WaaF_1.mol'
molecule_2=stk.BuildingBlock.init_from_file(filename)
molecule_2=molecule_2.with_centroid([0,0,0])
show_stk_mol(molecule_2)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
list_of_vertices_2,vertice_size_2 = Optimiser_functions.Axes.BySmiles(molecule_2,smiles_string='[Rh]')
list_of_vertices_filt_2=Optimiser_functions.Axes.remove_close_vectors(list_of_vertices_2)
facet_axes_2,centroid_to_facet_2 =Optimiser_functions.Axes.ByMidpoint(molecule_2,
    vectors=list_of_vertices_filt_2,
    vertice_size=vertice_size_2,
    no_vectors_define_facet=3,
    tolerance=0.1)

list_of_edges_2,centroid_to_edge_2 = Optimiser_functions.Axes.BySmiles(molecule_2,smiles_string=utils.remove_aldehyde('O=Cc1cc(C=O)cc(C=O)c1'))

list_of_windows_2 = facet_axes_2
centroid_to_window_2 = centroid_to_facet_2


[[ 0.57735046  0.57734284  0.57735751]
 [ 0.5773385  -0.57735994  0.57735237]
 [-0.57735513  0.5773405   0.57735518]
 [-0.57735225 -0.57735306  0.57734549]
 [ 0.57735282  0.57735202 -0.57734596]
 [ 0.57734995 -0.57735095 -0.5773499 ]
 [-0.57734842  0.57735422 -0.57734817]
 [-0.57735462 -0.57734862 -0.57734757]]


In [8]:
mol_with_vec(molecule_2,list_of_windows_2,centroid_to_window_2)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now we have the axes defined, you can then use the code to generate the dimers. For this you will need the following information:

    molecule: The cage you want to displace from
    molecule_2: The second cage you want to calculate the interaction between
    axes_1: The axis you want to displace along from the orginal cage
    axes_2: The axis of the second cage you want facing the first cage
    displacement_distance: float. This is the starting distance of the displacement between the two molecules. (It actually starts 2 angstroms closer)
    displacement: float. This+displacement_distance is the maximum distance of displacement
    displacement_step_size: float. This is the step size of the displacement.
    rotation_limit: float. This is the maximum rotation of the displaced cage.
    rotation_step_size: float. This is the step size of the rotation.
    slide: bool. This is whether you want translating ove the face or not
    
and it returns a dictorary with the following:

    'Displacement shell': The "shell" of the displacement, i.e. which "displacement" value you're doing
    'Slide': Which position along the face you're at if you're sliding, doesnt actually equate to where but 0 is on axis and not 0 is off axis
    'Rotation': The rotation of the cage
    'Displacement centroid': The actual displacement of the centroid of the neighbouring cage
    'Dimer': the stk.Molecule of the dimer


In [9]:
list_of_dimers = dimer_calculations.DimerGenerator.generate(molecule,
        molecule_2= molecule,
        axes_1=list_of_windows[0],
        axes_2=list_of_windows[0],
        displacement_distance=centroid_to_window+centroid_to_window-2,
        displacement=10,
        displacement_step_size=1,
        rotation_limit=120,
        rotation_step_size=30,
        slide=False,
        )

[-0.0072742  -0.94544196  0.32570935]
[0.57735046 0.57734284 0.57735751]


Play around with the show_stk_mol to have a look at what the different dimers look like.

In [10]:
show_stk_mol(list_of_dimers[0]['Dimer'])
#mol_with_vec(list_of_dimers[0]['Dimer'],list_of_windows,centroid_to_window_2)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Can you see that for some of them the cages are overlapping? We don't want to optimise these dimers as they could lead to catenated cages (which we aren't interested in) or the cages would be very high in energy (which again we aren't interested in. Therefore we usually check if the dimer has an overlap between the cages before relaxing the structure. We can also check whether the cages are catenated and not optimise catenated structures

In [11]:
dimers_for_optimisation=[]
print(len(list_of_dimers))
for dimer_entry in list_of_dimers:
    print()
    if not os.path.exists('dimers'):
        os.makedirs('dimers')
    filename = f"dimers/Shell_{dimer_entry['Displacement shell']}_slide_dimer_entry{dimer_entry['Slide']}_rot_{dimer_entry['Rotation']}.mol"
    dimer_entry['Dimer'].write(filename)
    rdkit_molecule = Chem.MolFromMolFile(filename)
    overlap=dimer_calculations.check_overlaps(rdkit_molecule, threshold=1)
    if not overlap:
        catenated=check_catenation(filename,molecule)
        if not catenated:
            dimers_for_optimisation.append(dimer_entry)
print(len(dimers_for_optimisation))


40








































33


Now we're ready for optimisation. There are a few different options we have in how we're going to optimise. The first is whether we're going to keep the relative positions of the cages/molecules constant. This is important if you're trying to figure out the depth of the potential energy well of the different packing motifs. For some softwares, it wont matter as the optimisation is unlikely going to make a big difference to the relative positions (e.g. DFT calculations of semi-emprical calculations), but for others there can be a large reorganisation of the molecules. To avoid this we can "fix" the atomic positions of some atoms in the cage using the fix_atom_set below. This can fix any atoms you input, but we tend to do it by the nitrogen positions in the imines (for POCs) and the metal_atom positions (for MOCs) as these usually define the shapes of the cages as well

In [12]:
fixed_atom_set = cage.Cage.fix_atom_set(
    list_of_dimers[0]['Dimer'],
    'NCCN',
    metal_atom=None
)

The other thing to consider is what optimiser you would like to use for calculating the energy of the dimers of molecules. The optimiser you choose will depend on (1) the type of molecule you are looking at and (2) the level of accuracy you require. There are three choices: 

    1. OPLS
    2. GULP
    3. XTB
    
For POCs we typically choose OPLS (soon to be replaced by openMM) and for MOCs we typically use XTB. Depending on which software you use will determine how you then collect the energies at then. As those in the group who will use this will be using openMM, we won't go any further in this tutorial for now.

In [ ]:
for dimer_entry in dimers_for_optimisation:
    dimer_calculations.DimerOptimizer.optimise_dimer_XTB(dimer_entry['Dimer'],f"XTB_shell_{dimer_entry['Displacement shell']}_slide_{dimer_entry['Slide']}_rot_{dimer_entry['Rotation']}",XTB_PATH,opt_level='crude',num_cores=4,unpaired_electrons=24,fixed_atom_set=None)